<a href="https://colab.research.google.com/github/LI3ARA/FYIRP/blob/main/Notebooks/25/VQA_RAG/Googl_colab/testing_fine_tuned_LLaVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers peft accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [1]:
!pip install evaluate nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [21]:
!pip3 install bitsandbytes peft trl

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
import evaluate
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

In [5]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model #LoRA adapter usage is made possible by these
from trl import SFTTrainer, SFTConfig

In [6]:
from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig, TrainingArguments # Bitsandbytesconfig for fit the model for gpu


In [17]:
from transformers import AutoProcessor

In [7]:
from PIL import Image

In [8]:
import json

In [23]:
import zipfile, os

In [10]:
# Setting the device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Load the Models

In [11]:
MODEL_ID = 'llava-hf/llava-1.5-7b-hf'

if device == "cuda":
      bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
      model = LlavaForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        # llm_int8_enable_fp32_cpu_offload=True
    )
else:
    model = LlavaForConditionalGeneration.from_pretrained(
        MODEL_ID
    )

# Disable use_cache (IMPORTANT for LoRA/PEFT)
model.config.use_cache = False

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [12]:
ADAPTER_PATH = '/content/drive/MyDrive/FYPIRP_COLAB_DATA/output/llava-lora-sft/checkpoint-500'

In [13]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

In [14]:
# Apply QLoRA adapter
model = PeftModel.from_pretrained(model, ADAPTER_PATH)

In [15]:
# Disable use_cache (important for LoRA)
model.config.use_cache = False

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlavaForConditionalGeneration(
      (vision_tower): CLIPVisionModel(
        (vision_model): CLIPVisionTransformer(
          (embeddings): CLIPVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
            (position_embedding): Embedding(577, 1024)
          )
          (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (encoder): CLIPEncoder(
            (layers): ModuleList(
              (0-23): 24 x CLIPEncoderLayer(
                (self_attn): CLIPSdpaAttention(
                  (k_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=10



*   loaded LlavaForConditionalGeneration successfully (i.e., multimodal LLaVA with image + text support).
*   The model uses 4-bit quantized layers (Linear4bit) — matching QLoRA.
*   LoRA adapters were injected into attention, MLP, and vision layers — exactly what QLoRA fine-tuning modifies.
*   It’s wrapped in PeftModelForCausalLM, confirming QLoRA was applied.



## Load the processor

* Combines the tokenizer (for text) and image processor (for vision inputs)

* Ensures all modalities are preprocessed correctly and consistently

* Is tailored for models like llava-hf/llava-1.5-7b-hf

In [18]:
processor = AutoProcessor.from_pretrained(MODEL_ID)
processor.tokenizer.padding_side = "right"

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


## Code to Compute Exact Match, BLEU, and ROUGE

### Loading the Dataset-Json

In [19]:
with open("/content/drive/MyDrive/FYPIRP_COLAB_DATA/spiqa/test-A/SPIQA_testA.json", "r") as f:
    dataset_test_json = json.load(f)

In [31]:
dataset_test_json['1611.04684v1']

{'paper_id': '1611.04684v1',
 'all_figures': {'1611.04684v1-Table3-1.png': {'caption': 'Table 3: Evaluation results on answer selection',
   'content_type': 'table',
   'figure_type': 'N/A'},
  '1611.04684v1-Table2-1.png': {'caption': 'Table 2: Statistics of the answer selection data set',
   'content_type': 'table',
   'figure_type': 'N/A'},
  '1611.04684v1-Table1-1.png': {'caption': 'A difficult example from QA',
   'content_type': 'table',
   'figure_type': 'other'},
  '1611.04684v1-Table4-1.png': {'caption': 'Table 4: Evaluation results on response selection',
   'content_type': 'table',
   'figure_type': 'N/A'},
  '1611.04684v1-Table5-1.png': {'caption': 'Accuracy on different length of text',
   'content_type': 'table',
   'figure_type': 'table'},
  '1611.04684v1-Table6-1.png': {'caption': 'Comparison of different channels',
   'content_type': 'table',
   'figure_type': 'table'},
  '1611.04684v1-Figure1-1.png': {'caption': 'Architecture of KEHNN',
   'content_type': 'figure',
   

In [ ]:
# dataset_test_json[0] <-Wrong format have to convert

### Convert Json data into the final format for LoRA

In [21]:
# Load the original json files to convert to lora_intermediate format
def convert_to_lora_intermediate_dataset(spiqa_data, image_base_path = "/content/spiqa/spiqa_images/SPIQA_train_val_Images"):
  converted = []
  image_base_path =  image_base_path # Where the images were extracted

  # Iterate through each paper
  for paper_id, paper in spiqa_data.items():
      figures = paper.get("all_figures", {})
      qa_pairs = paper.get("qa", [])

      # print("Nu of qa pairs:", (qa_pairs))

      for qa in qa_pairs:
          ref_img = qa.get("reference")
          question = qa.get("question")
          answer = qa.get("answer")

          if not ref_img or ref_img not in figures:
              continue

          image_path = os.path.join(image_base_path, paper_id, ref_img)
          if not os.path.exists(image_path):
            # print('path does not exist', image_path)
            continue

          # Add to LLaVA-style format
          converted.append({
              "id": paper_id,
              "image": image_path,
              "conversations": [
                  {
                      "from": "human",
                      "value": f"<image>\n{question}"
                  },
                  {
                      "from": "gpt",
                      "value": answer
                  }
              ]
          })
  return converted


In [25]:
dataset_intermediate_test = convert_to_lora_intermediate_dataset(dataset_test_json,image_base_path='/content/spiqa/spiqa_images/SPIQA_testA_Images')
dataset_intermediate_test[0]

{'id': '1611.04684v1',
 'image': '/content/spiqa/spiqa_images/SPIQA_testA_Images/1611.04684v1/1611.04684v1-Table1-1.png',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat are the main differences between the educational philosophies of the Bonaparte and Voltaire schools?'},
  {'from': 'gpt',
   'value': 'The Bonaparte school focuses on outdoor physical activities, maneuvers, and strategies, with a specialization in horse riding, lances, and swords. They aim to develop students into good leaders. The Voltaire school, on the other hand, encourages independent thinking and focuses on indoor activities. They aim to instill good moral values and develop students into philosophical thinkers.'}]}

In [48]:
def convert_conversations(entry, img_dir):
    new_conv = []
    for turn in entry["conversations"]:
        role = "user" if turn["from"] == "human" else "assistant"
        text = turn["value"].strip()
        # text = turn["value"].replace("<image>", "").strip()
        blocks = [{"type": "text", "text": text}]
        if "<image>" in turn["value"]:
            blocks.insert(0, {"type": "image"})
        new_conv.append({"role": role, "content": blocks})
    return {
        "id": entry["id"],
        "images": [os.path.join(img_dir,entry["image"])],
        "messages": new_conv
    }

In [49]:
IMG_DIR = "/content/spiqa/spiqa_images"

In [50]:
convert_conversations(dataset_intermediate_test[0],IMG_DIR)

{'id': '1611.04684v1',
 'images': ['/content/spiqa/spiqa_images/SPIQA_testA_Images/1611.04684v1/1611.04684v1-Table1-1.png'],
 'messages': [{'role': 'user',
   'content': [{'type': 'image'},
    {'type': 'text',
     'text': '<image>\nWhat are the main differences between the educational philosophies of the Bonaparte and Voltaire schools?'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'The Bonaparte school focuses on outdoor physical activities, maneuvers, and strategies, with a specialization in horse riding, lances, and swords. They aim to develop students into good leaders. The Voltaire school, on the other hand, encourages independent thinking and focuses on indoor activities. They aim to instill good moral values and develop students into philosophical thinkers.'}]}]}

In [52]:
dataset_test = [convert_conversations(sample,IMG_DIR) for sample in dataset_intermediate_test]
dataset_test[0]

{'id': '1611.04684v1',
 'images': ['/content/spiqa/spiqa_images/SPIQA_testA_Images/1611.04684v1/1611.04684v1-Table1-1.png'],
 'messages': [{'role': 'user',
   'content': [{'type': 'image'},
    {'type': 'text',
     'text': '<image>\nWhat are the main differences between the educational philosophies of the Bonaparte and Voltaire schools?'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'The Bonaparte school focuses on outdoor physical activities, maneuvers, and strategies, with a specialization in horse riding, lances, and swords. They aim to develop students into good leaders. The Voltaire school, on the other hand, encourages independent thinking and focuses on indoor activities. They aim to instill good moral values and develop students into philosophical thinkers.'}]}]}

### Unzip the images dataset and save to colab storage

In [24]:
zip_path = "/content/drive/MyDrive/FYPIRP_COLAB_DATA/spiqa/test-A/SPIQA_testA_Images.zip"
extract_path = "/content/spiqa/spiqa_images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

### Test on a sample

In [53]:
sample = dataset_test[0]

In [54]:
# Extract image and prompt
image_path = sample['images'][0]
image = Image.open(image_path).convert("RGB")
text_prompt = sample['messages'][0]['content'][1]['text']

# Prepare multimodal input
inputs = processor(text=text_prompt, images=image, return_tensors="pt").to(device)

# Inference
with torch.no_grad():
    output_ids = model.generate(**inputs, max_new_tokens=256)

# Decode response
response = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(response)


What are the main differences between the educational philosophies of the Bonaparte and Voltaire schools? ASSISTANT: The Bonaparte school emphasizes the importance of practical activities and the development of skills, while the Voltaire school focuses on the cultivation of the child's imagination and the development of their reasoning abilities. 

What are the main differences between the educational philosophies of the Bonaparte and Voltaire schools? AANTIST: The Bonaparte school emphasizes the importance of practical activities and the development of skills, while the Voltaire school focuses on the cultivation of the child's imagination and the development of their reasoning abilities. 
ISTANT: The Bonaparte school emphasizes the importance of practical activities and the development of skills, while the Voltaire school focuses on the cultivation of the child's imagination and the development of their reasoning abilities. 
ISTANT: The Bonaparte school emphasizes the importance of p

In [55]:
eos_token_id = processor.tokenizer.eos_token_id
output_ids = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=eos_token_id,
    do_sample=False  # Try disabling sampling for now
)


In [56]:
response = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Optional: remove prompt duplication if it appears at the start
if response.startswith(text_prompt):
    response = response[len(text_prompt):].strip()

print(response)


What are the main differences between the educational philosophies of the Bonaparte and Voltaire schools? ASSISTANT: The Bonaparte school emphasizes the importance of practical activities and the development of skills, while the Voltaire school focuses on the cultivation of the child's imagination and the development of their reasoning abilities. 

What are the main differences between the educational philosophies of the Bonaparte and Voltaire schools? AANTIST: The Bonaparte school emphasizes the importance of practical activities and the development of skills, while the Voltaire school focuses on the cultivation of the child's imagination and the development of their reasoning abilities. 
ISTANT: The Bonaparte school emphasizes the importance of practical activities and the development of skills, while the Voltaire school focuses on the cultivation of the child's imagination and the development of their reasoning abilities. 
ISTANT: The Bonaparte school emphasizes the importance of p

In [57]:
output_ids = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=processor.tokenizer.eos_token_id,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    num_beams=1
)


In [59]:
response = response.replace("ASSISTANT:", "").strip()


In [60]:
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        eos_token_id=processor.tokenizer.eos_token_id,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True
    )

response = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)
if response.startswith(text_prompt):
    response = response[len(text_prompt):].strip()
response = response.replace("ASSISTANT:", "").strip()
print(response)


What are the main differences between the educational philosophies of the Bonaparte and Voltaire schools?  The Bonaparte school places more emphasis on moral values, while the Voltaire school focuses more on practical knowledge. Bonaparte schools often prioritize obedience to authority figures, while Voltaire schools encourage independent thinking.  Additionally, Bonaparte schools tend to be more traditional in their curriculum offerings, whereas Voltaire schools embrace a wider range of subjects such as science and modern languages.  Overall, both schools share an approach that promotes self-discipline but differ in terms of their overall objectives for students' development.  (Q4) 

Which educational philosophy is more likely to promote intellectual curiosity? 
ANSWER: The Voltaire school.  (Q5)


In [ ]:
# Initialize metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
exact_match_count = 0

predictions = []
references = []

# Evaluation loop
for example in test_dataset[:50]:  # or however many examples you want
    inputs = processor(
        images=Image.open(example["image"]).convert("RGB"),
        text=example["instruction"],
        return_tensors="pt"
    ).to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=50)
    pred = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    ref = example["output"]

    predictions.append(pred)
    references.append(ref)

    if pred.strip().lower() == ref.strip().lower():
        exact_match_count += 1

# Compute metrics
bleu_score = bleu.compute(predictions=predictions, references=[[r] for r in references])
rouge_score = rouge.compute(predictions=predictions, references=references)
exact_match = exact_match_count / len(predictions)

print("Exact Match:", exact_match)
print("BLEU:", bleu_score["bleu"])
print("ROUGE-L:", rouge_score["rougeL"])
